<img
src='https://static1.squarespace.com/static/5856f62ed1758e23c117f905/t/5c95a9fdf9619a337b7776e9/1553312289653/data-science-spring-texas.png?format=1000w' align='left'> 

# IBM Capstone Project by Valentina Fontiveros

On this Jupyter Notebook, I will document my progress on the IBM Data Science Capstone Project.

In [1]:
# First, let's import some basic libraries

import pandas as pd
import numpy as np

# Now, let's print a silly message in order to follow the rules of this assignment.

print('Hello, Capstone Project Course!')

Hello, Capstone Project Course!
